In [1]:
import sys
import os
sys.path.append("../../")
#sys.path.append("workspace/code/Nahush26/Tmet")


In [2]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.datasets.classification import DatasetFactory

In [3]:
from trailmet.algorithms.prune.network_slimming_train import NetworkSlimming

In [4]:
data_root = os.getcwd()


In [5]:
train_transform=transforms.Compose([
                                      transforms.Pad(4),
                                      transforms.RandomCrop(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                  ])
val_transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ])
test_transform = val_transform

In [6]:
transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}
cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test_dataset'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )
dataloader = {"train" : train_loader , "test" : val_loader}

In [8]:
import yaml
import os
with open(os.path.join(data_root,"cifar10_ns.yml"),'r') as stream:
    data_loaded = yaml.safe_load(stream)

In [9]:
data_loaded

{'NETWORK_SLIMMING': {'DATA': 'cifar10',
  'ARCH': 'vgg',
  'NUM_CLASSES': 10,
  'SPARSITY_REG': True,
  'THR': 1e-05,
  'FINE_TUNE': False,
  'RESUME': False,
  'TRAIN_BS': 64,
  'TEST_BS': 256,
  'EPOCHS': 100,
  'OPTIMIZER_NAME': 'SGD',
  'LR': 0.1,
  'MOMENTUM': 0.9,
  'WEIGHT_DECAY': 0.0001,
  'LOG_INTERVAL': 100,
  'DEPTH': 164,
  'PERCENT': 0.6}}

In [10]:
class CFG:
    def __init__(self,train_loader = train_loader , val_loader = val_loader):
        self.DATA = 'cifar10'
        #self.epochs = 256
        #self.lr = 0.001
        #self.momentum = 0.9
        #self.weight_decay = 0
        self.TRAIN_LOADER = train_loader
        self.TEST_LOADER = val_loader
        self.OPTIMIZER_NAME = "SGD"
        self.FINE_TUNE = True
        self.SPARSITY_REG = False
        self.PATH = './pruned_checkpoint.pth.tar'
        self.SAVE ='./pruned_checkpoint.pth.tar'
        self.MODEL = 'model_best.pth.tar'
    
        #self.label_smooth = 0.1
        #self.num_workers = 4
        #self.dataset = 'cifar10'  #valid options are cifar100, cifar10 and tiny-imagenet
        #self.num_classes = 10 #10 for CIFAR10 // 100 for CIFAR100 // 200 for Tiny ImageNet 

In [11]:
check = NetworkSlimming(data_loaders  = dataloader ,path = './pruned_checkpoint.pth.tar' , save = './pruned_checkpoint.pth.tar', model = 'model_best.pth.tar' , **data_loaded)

In [12]:
model1 = check.base_line()

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:03<00:00, 45.26it/s, acc=0.555, loss=1.22]


Train set :: Average loss: 1.5512051419258117 


Test set: Average loss: 1.215915280542556, Accuracy: 55.55 

1


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:03<00:00, 44.57it/s, acc=0.676, loss=0.934]


Train set :: Average loss: 1.0408189326286317 


Test set: Average loss: 0.9338171717467582, Accuracy: 67.62 

2


 59%|█████████████████████████████████████████████████████████████▉                                           | 369/625 [00:14<00:10, 25.26it/s, loss=0.848]


KeyboardInterrupt: 